In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# author:Xavier

import tkinter
import tkinter as tk
import numpy as np
import pandas as pd
from tkinter import *
from tkinter import filedialog
from tkinter import ttk

import info_gain
import preprocess
import quality_predict
import parameter_optimize

def read_file():
    file_name = filedialog.askopenfilename(title="选择数据集", filetypes=[("S2out", "*.csv"), ("All Files", "*")])
    raw_df = preprocess.GUI_csv_to_df(file_name)
    for index, row in raw_df.iterrows():
        treeview.insert('', index, values=(index, row['moment'], row["flatness"], row["label"]))

window = tkinter.Tk()
window.title('工艺参数优化工具集')
window.geometry('500x550')

frame_title = Frame(window)
frame_table = Frame(window)

canvas = tkinter.Canvas(frame_title, width=130, height=130, bg='white')
image_file = tkinter.PhotoImage(file='buaa.gif')
image = canvas.create_image(0, 0, anchor='nw', image=image_file)
canvas.pack(side=LEFT)
tkinter.Label(frame_title, text='工艺参数优化工具集',font=('宋体', 26)).pack(side=LEFT, padx=20)

tkinter.Label(frame_table, text='原始数据读取',font=('宋体', 20)).pack(side=TOP)
btn1 = Button(frame_table, text="读取数据集", font =("宋体",20,'bold'), width=8, height=1, command=read_file)
btn1.pack(side=TOP)
columns = ("编号", "夹紧力矩", "底面平面度", "质量标签")
treeview = ttk.Treeview(frame_table, height=13, show="headings", columns=columns)
treeview.column("编号", width=100, anchor='center')
treeview.column("夹紧力矩", width=100, anchor='center')
treeview.column("底面平面度", width=100, anchor='center')
treeview.column("质量标签", width=100, anchor='center')
treeview.heading("编号", text="编号")
treeview.heading("夹紧力矩", text="夹紧力矩")
treeview.heading("底面平面度", text="底面平面度")
treeview.heading("质量标签", text="质量标签")
treeview.pack(side=TOP, fill=X)

frame_title.pack(anchor=NW)
frame_table.pack(anchor=NW, fill=X)

def calc_info_gain():  
    def close_window():
        window_infoGain.destroy()
    def vis_infoGain():
        for index, row in enumerate(columns_entropy_list):
            treeview.insert('', index, values=(row[0], row[1]))        
        
    window_infoGain = tk.Toplevel(window)
    window_infoGain.geometry('400x120')
    window_infoGain.title('工艺参数优先级排序')
    
    file_path = './dataset/moment_table.csv'
    raw_df = preprocess.GUI_csv_to_df(file_path)
    
    columns_entropy = [(col, info_gain.calcu_each_gain(raw_df[col], raw_df)) for col in raw_df.iloc[:, :-1]]
    
    frame_result = Frame(window_infoGain)
    frame_btn = Frame(window_infoGain)
    
    textLabel = Label(frame_result,
                      text="信息增益计算结果",
                      justify=LEFT)
    textLabel.pack(side=LEFT)    
    
    columns = ("工艺参数", "信息增益")
    treeview = ttk.Treeview(frame_result, height=2, show="headings", columns=columns)
    treeview.column("工艺参数", width=100, anchor='center')
    treeview.column("信息增益", width=100, anchor='center')

    treeview.heading("工艺参数", text="工艺参数")
    treeview.heading("信息增益", text="信息增益")
    treeview.pack()
    
    columns_entropy_list = []
    cell1 = []
    cell1.append("夹紧力矩")
    cell1.append(columns_entropy[0][1])
    cell2 = []
    cell2.append("底面平面度")
    cell2.append(columns_entropy[1][1])
    columns_entropy_list.append(cell1)
    columns_entropy_list.append(cell2)
    
    btn1 = Button(frame_btn, text="计算信息增益", font =("宋体",20,'bold'), width=12, height=1, command=vis_infoGain)
    btn1.pack(side=TOP)
    btn2 = Button(frame_btn, text="返回", font =("宋体",20,'bold'), width=8, height=1, command=close_window)
    btn2.pack(side=TOP)
    
    frame_result.pack()
    frame_btn.pack()

def calc_quality_predict():
    def close_window():
        window_qualityPredict.destroy()
        
    def training_model():
        file_path = './dataset/moment_table.csv'
        training_df = preprocess.GUI_csv_to_df(file_path)
        feature_matrix, feature_matrix_mean, feature_matrix_std = preprocess.feature_eng(training_df)
        label_matrix = preprocess.label_eng(training_df)
        generator = quality_predict.cv_generator(feature_matrix, label_matrix)
        
        model_index = v.get()
        if model_index == 1:
            running_model = quality_predict.model_LR()
        elif model_index == 2:
            running_model = quality_predict.model_SVM()
        elif model_index ==3:
            running_model = quality_predict.model_XGBoost()
        else:
            pass
        
        nodes = treeview.get_children()
        for item in nodes:
            treeview.delete(item)
        
        visual_str = quality_predict.GUI_frame_classification(generator,
                                                              running_model,
                                                              feature_matrix,
                                                              label_matrix,
                                                              "./model_output/",
                                                              treeview)
        
    def vis_pred_data():
        file_name = filedialog.askopenfilename(title="选择数据集", filetypes=[("S2out", "*.csv"), ("All Files", "*")])
        print(file_name)
        raw_df = preprocess.GUI_csv_to_df(file_name)
        for index, row in raw_df.iterrows():
            treeview_pred.insert('', index, values=(index, row['moment'], row["flatness"], row["label"]))

    def pred_quality():
        feature_matrix, pre_df = quality_predict.GUI_pred_sample_reader(file_name)
        load_model_dir = "./model_output/" + "model_1"
        pred_result = quality_predict.GUI_qual_pred(load_model_dir, feature_matrix)
        pre_df['label'] = pred_result
        
        for index, row in pre_df.iterrows():
            treeview_result.insert('', index, values=(index, row['moment'], row["flatness"], row["label"]))
        
    #file_name = filedialog.askopenfilename(title="选择数据集", filetypes=[("S2out", "*.csv"), ("All Files", "*")])
    file_name = "/Users/xuejingyuan/Desktop/Github/Process-Parameter-Optimization/dataset/moment_pred.csv"
    
    ## 1.模型选择
    window_qualityPredict = tk.Toplevel(window)
    window_qualityPredict.geometry('530x620')
    window_qualityPredict.title('零件质量预测')
    
    frame_model = Frame(window_qualityPredict)
    
    group = LabelFrame(frame_model, text='选择预测模型', font=('宋体', 20))
    group.pack(side=LEFT)
    
    models = [('LR', 1), ('SVM', 2), ('XGBoost', 3), ('GBDT + LR', 4)]
    v = IntVar()
    v.set(1)
    
    for model, num in models:
        b = Radiobutton(group, text=model, font=('宋体', 15), variable=v, value=num)
        b.pack(side=LEFT)

    btn1 = Button(frame_model, text="训练模型", font =("宋体",20,'bold'), width=8, height=1, command=training_model)
    btn1.pack(side=LEFT, padx=10, pady=20)
    
    ## 2.模型训练    
    frame_accuracy = Frame(window_qualityPredict)

    textLabel = Label(frame_accuracy,
                      text="模型准确率",
                      font =("宋体",20,'bold'),
                      justify=LEFT)
    textLabel.pack(side=TOP)
        
    columns = ("十折交叉验证索引", "正确分类样本量", "样本总数", "模型准确率")
    treeview = ttk.Treeview(frame_accuracy, height=11, show="headings", columns=columns)
    treeview.column("十折交叉验证索引", width=100, anchor='n')
    treeview.column("正确分类样本量", width=100, anchor='n')
    treeview.column("样本总数", width=100, anchor='n')
    treeview.column("模型准确率", width=120, anchor='n')
    treeview.heading("十折交叉验证索引", text="十折交叉验证索引")
    treeview.heading("正确分类样本量", text="正确分类样本量")
    treeview.heading("样本总数", text="样本总数")
    treeview.heading("模型准确率", text="模型准确率")
    treeview.pack(side=TOP)
        
    ## 3.预测零件质量
    frame_read = Frame(window_qualityPredict)
    
    textLabel = Label(frame_read,
                      text="预测零件数据",
                      font =("宋体",20,'bold'),
                      justify=LEFT)
    textLabel.pack(side=LEFT)
    
    btn2 = Button(frame_read, text="读取预测数据", font =("宋体",20,'bold'), width=12, height=1, command=vis_pred_data)
    btn2.pack(side=LEFT)
    
    frame_read_table = Frame(window_qualityPredict)
    
    columns = ("编号", "夹紧力矩", "底面平面度", "原始质量标签")
    treeview_pred = ttk.Treeview(frame_read_table, height=5, show="headings", columns=columns)
    treeview_pred.column("编号", width=100, anchor='center')
    treeview_pred.column("夹紧力矩", width=100, anchor='center')
    treeview_pred.column("底面平面度", width=100, anchor='center')
    treeview_pred.column("原始质量标签", width=100, anchor='center')

    treeview_pred.heading("编号", text="编号")
    treeview_pred.heading("夹紧力矩", text="夹紧力矩")
    treeview_pred.heading("底面平面度", text="底面平面度")
    treeview_pred.heading("原始质量标签", text="原始质量标签")
    treeview_pred.pack()
    
    frame_predict = Frame(window_qualityPredict)

    textLabel = Label(frame_predict,
                      text="零件质量预测结果",
                      font =("宋体",20,'bold'),
                      justify=LEFT)
    textLabel.pack(side=LEFT)
    
    btn3 = Button(frame_predict, text="预测零件质量", font =("宋体",20,'bold'), width=12, height=1, command=pred_quality)
    btn3.pack(side=LEFT)
    
    frame_predict_table = Frame(window_qualityPredict)
    columns = ("编号", "夹紧力矩", "底面平面度", "预测结果")
    treeview_result = ttk.Treeview(frame_predict_table, height=5, show="headings", columns=columns)
    treeview_result.column("编号", width=100, anchor='center')
    treeview_result.column("夹紧力矩", width=100, anchor='center')
    treeview_result.column("底面平面度", width=100, anchor='center')
    treeview_result.column("预测结果", width=100, anchor='center')

    treeview_result.heading("编号", text="编号")
    treeview_result.heading("夹紧力矩", text="夹紧力矩")
    treeview_result.heading("底面平面度", text="底面平面度")
    treeview_result.heading("预测结果", text="预测结果")
    treeview_result.pack()
    

    frame_model.pack(side=TOP)
    frame_accuracy.pack(side=TOP)
    frame_read.pack(side=TOP)
    frame_read_table.pack(side=TOP)
    frame_predict.pack(side=TOP)
    frame_predict_table.pack(side=TOP)
    frame_predict.pack(side=TOP)

def calc_optimize_parameter():
    def read_optFile():
        file_name = filedialog.askopenfilename(title="选择数据集", filetypes=[("S2out", "*.csv"), ("All Files", "*")])
        raw_df = preprocess.GUI_csv_to_df(file_name)
        for index, row in raw_df.iterrows():
            treeview.insert('', index, values=(index, row["flatness"], row["label"], row["moment"]))
    def opt_param():
        opt_train_file_name = "./dataset/moment_table.csv"
        feature_matrix, feature_mean, feature_std, feature_label = parameter_optimize.GUI_op_training_data(opt_train_file_name)
        opt_feature, opt_feature_df = parameter_optimize.GUI_op_sample_reader(file_name, feature_mean, feature_std)
        opt_result = parameter_optimize.GUI_para_optimize(feature_matrix, feature_label, opt_feature)
        opt_feature_df["moment"] = opt_result
        
        for index, row in opt_feature_df.iterrows():
            treeview_result.insert('', index, values=(index, row['flatness'], "37.4", row["moment"]))
            
    file_name = "/Users/xuejingyuan/Desktop/Github/Process-Parameter-Optimization/dataset/moment_opt.csv"
    window_paramOpt = tk.Toplevel(window)
    window_paramOpt.geometry('600x300')
    window_paramOpt.title('工艺参数优化')
    
    frame_raw_label = Frame(window_paramOpt)
    frame_raw = Frame(window_paramOpt)
    
    textLabel = Label(frame_raw_label,
                      text="未优化工艺参数",
                      font =("宋体",20,'bold'),
                      justify=LEFT)
    textLabel.pack(side=LEFT)
    
    btn2 = Button(frame_raw_label, text="读取优化数据", font =("宋体",20,'bold'), width=13, height=1, command=read_optFile)
    btn2.pack(side=LEFT)
    
    columns = ("编号", "底面平面度", "零件质量标签", "夹紧力矩")
    treeview = ttk.Treeview(frame_raw, height=5, show="headings", columns=columns)
    treeview.column("编号", width=100, anchor='center')
    treeview.column("底面平面度", width=100, anchor='center')
    treeview.column("零件质量标签", width=100, anchor='center')
    treeview.column("夹紧力矩", width=100, anchor='center')

    treeview.heading("编号", text="编号")
    treeview.heading("底面平面度", text="底面平面度")
    treeview.heading("零件质量标签", text="零件质量标签")
    treeview.heading("夹紧力矩", text="夹紧力矩")
    treeview.pack(side=TOP)
 
    frame_result_label = Frame(window_paramOpt)
    textLabel = Label(frame_result_label,
                      text="工艺参数优化结果",
                      font =("宋体",20,'bold'),
                      justify=LEFT)
    textLabel.pack(side=LEFT)
    btn1 = Button(frame_result_label, text="优化工艺参数", font =("宋体",20,'bold'), width=13, height=1, command=opt_param)
    btn1.pack(side=LEFT)
    
    frame_result = Frame(window_paramOpt)

    columns = ("编号", "底面平面度", "零件质量标签", "夹紧力矩")
    treeview_result = ttk.Treeview(frame_result, height=5, show="headings", columns=columns)
    treeview_result.column("编号", width=100, anchor='center')
    treeview_result.column("底面平面度", width=100, anchor='center')
    treeview_result.column("零件质量标签", width=100, anchor='center')
    treeview_result.column("夹紧力矩", width=100, anchor='center')

    treeview_result.heading("编号", text="编号")
    treeview_result.heading("底面平面度", text="底面平面度")
    treeview_result.heading("零件质量标签", text="零件质量标签")
    treeview_result.heading("夹紧力矩", text="夹紧力矩")
    treeview_result.pack(side=TOP)
    
    frame_raw_label.pack(side=TOP)
    frame_raw.pack(side=TOP)
    frame_result_label.pack(side=TOP)
    frame_result.pack(side=TOP)
    
frame_button = Frame(window)

btn_info_gain = tk.Button(frame_button, text='工艺参数优先级排序', font =("宋体",20,'bold'), width=14, height=1, command=calc_info_gain)
btn_info_gain.pack(anchor=N)

btn_quality_predict = tk.Button(frame_button, text='零件质量预测', font =("宋体",20,'bold'), width=14, height=1, command=calc_quality_predict)
btn_quality_predict.pack(anchor=N)

btn_parameter_optimize = tk.Button(frame_button, text='工艺参数优化', font =("宋体",20,'bold'), width=14, height=1, command=calc_optimize_parameter)
btn_parameter_optimize.pack(anchor=N)

frame_button.pack(anchor=N)

# 第10步，主窗口循环显示
window.mainloop()

 *** Prediction model *** 
LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=20, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=1e-06, verbose=0, warm_start=False)
Writing trained model into dir : ./model_output/model_1
Writing trained model into dir : ./model_output/model_2
Writing trained model into dir : ./model_output/model_3
Writing trained model into dir : ./model_output/model_4
Writing trained model into dir : ./model_output/model_5
Writing trained model into dir : ./model_output/model_6
Writing trained model into dir : ./model_output/model_7
Writing trained model into dir : ./model_output/model_8
Writing trained model into dir : ./model_output/model_9
Writing trained model into dir : ./model_output/model_10
 *** Prediction model *** 
SVC(C=20, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto_de

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

 *** Prediction model *** 
XGBClassifier(alpha=2, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, eta=1, gamma=0,
       learning_rate=0.1, max_delta_step=1000, max_depth=4,
       min_child_weight=5, missing=None, n_estimators=200, n_jobs=1,
       nthread=None, num_boost_round=5, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=5,
       seed=None, silent=None, subsample=1, verbosity=1)
Writing trained model into dir : ./model_output/model_1
Writing trained model into dir : ./model_output/model_2
Writing trained model into dir : ./model_output/model_3
Writing trained model into dir : ./model_output/model_4
Writing trained model into dir : ./model_output/model_5
Writing trained model into dir : ./model_output/model_6
Writing trained model into dir : ./model_output/model_7
Writing trained model into dir : ./model_output/model_8
Writing trained model into dir : ./model_output/model_9
Wri